# Creación de subconjuntos

Descargamos los metadatos y etiquetas del conjunto ISIC 2019 para seleccionar 2 subconjuntos independientes balanceados por clase:
- Conjunto 1: 100 imágenes por cada clase para entrenamiento + 10 por cada clase para test
- Conjunto 2: 100 imágenes por cada clase para entrenamiento + 10 por cada clase para test

El dataset ISIC 2019 contiene 8 clases de lesiones de piel. Luego descargamos el archivo ZIP completo pero solo extraemos las imágenes seleccionadas, evitando problemas de espacio en el GitHub Codespace de 10GB.

## 1. Descarga de metadatos y etiquetas

In [1]:
import pandas as pd
import requests
from pathlib import Path

dest_dir = Path("conjunto_de_datos")
dest_dir.mkdir(parents=True, exist_ok=True)

csv_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Metadata.csv"
gt_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_GroundTruth.csv"

csv_path = dest_dir / "ISIC_2019_Training_Metadata.csv"
gt_path = dest_dir / "ISIC_2019_Training_GroundTruth.csv"

with requests.get(csv_url, timeout=60) as r:
    r.raise_for_status()
    csv_path.write_bytes(r.content)

with requests.get(gt_url, timeout=60) as r:
    r.raise_for_status()
    gt_path.write_bytes(r.content)

df_meta = pd.read_csv(csv_path)
df_gt = pd.read_csv(gt_path)

df = pd.merge(df_meta, df_gt, on='image')

print(f"Total de imágenes: {len(df)}")
print(f"Columnas: {list(df.columns)}\n")

clases = [col for col in df_gt.columns if col != 'image']
print(f"Clases de lesiones: {clases}\n")

for clase in clases:
    count = df[clase].sum()
    print(f"{clase}: {count} imágenes")

Total de imágenes: 25331
Columnas: ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex', 'MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

Clases de lesiones: ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

MEL: 4522.0 imágenes
NV: 12875.0 imágenes
BCC: 3323.0 imágenes
AK: 867.0 imágenes
BKL: 2624.0 imágenes
DF: 239.0 imágenes
VASC: 253.0 imágenes
SCC: 628.0 imágenes
UNK: 0.0 imágenes


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

clases = [col for col in df.columns if col not in ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex']]

df['clase'] = df[clases].idxmax(axis=1)

print(f"Distribución de clases:")
print(df['clase'].value_counts())

train1_dfs = []
test1_dfs = []
train2_dfs = []
test2_dfs = []

for clase in df['clase'].unique():
    df_clase = df[df['clase'] == clase]
    
    if len(df_clase) < 220:
        print(f"Advertencia: {clase} tiene solo {len(df_clase)} imágenes, usando todas disponibles")
        n_train = min(100, len(df_clase) // 2)
        n_test = min(10, (len(df_clase) - n_train) // 2)
    else:
        n_train = 100
        n_test = 10
    
    df_clase_shuffled = df_clase.sample(frac=1, random_state=42).reset_index(drop=True)
    
    set1 = df_clase_shuffled.iloc[:n_train + n_test]
    set2 = df_clase_shuffled.iloc[n_train + n_test:2*(n_train + n_test)]
    
    train1_dfs.append(set1.iloc[:n_train])
    test1_dfs.append(set1.iloc[n_train:n_train + n_test])
    
    train2_dfs.append(set2.iloc[:n_train])
    test2_dfs.append(set2.iloc[n_train:n_train + n_test])

train1_df = pd.concat(train1_dfs, ignore_index=True)
test1_df = pd.concat(test1_dfs, ignore_index=True)
train2_df = pd.concat(train2_dfs, ignore_index=True)
test2_df = pd.concat(test2_dfs, ignore_index=True)

train1_df.to_csv(dest_dir / "train1_subset.csv", index=False)
test1_df.to_csv(dest_dir / "test1_subset.csv", index=False)
train2_df.to_csv(dest_dir / "train2_subset.csv", index=False)
test2_df.to_csv(dest_dir / "test2_subset.csv", index=False)

print(f"\nConjunto 1 - Train: {len(train1_df)} imágenes")
print(train1_df['clase'].value_counts())
print(f"\nConjunto 1 - Test: {len(test1_df)} imágenes")
print(test1_df['clase'].value_counts())

print(f"\nConjunto 2 - Train: {len(train2_df)} imágenes")
print(train2_df['clase'].value_counts())
print(f"\nConjunto 2 - Test: {len(test2_df)} imágenes")
print(test2_df['clase'].value_counts())

train1_imgs = set(train1_df['image'])
test1_imgs = set(test1_df['image'])
train2_imgs = set(train2_df['image'])
test2_imgs = set(test2_df['image'])

all_sets = [
    ("train1", train1_imgs),
    ("test1", test1_imgs),
    ("train2", train2_imgs),
    ("test2", test2_imgs)
]

has_overlap = False
for i, (name1, set1) in enumerate(all_sets):
    for name2, set2 in all_sets[i+1:]:
        overlap = set1 & set2
        if overlap:
            print(f"ERROR: {len(overlap)} imágenes repetidas entre {name1} y {name2}")
            has_overlap = True

if not has_overlap:
    print("\n✓ No hay solapamiento entre conjuntos")

selected_images = train1_imgs | test1_imgs | train2_imgs | test2_imgs
print(f"\nTotal de imágenes a descargar: {len(selected_images)}")

Distribución de clases:
clase
NV      12875
MEL      4522
BCC      3323
BKL      2624
AK        867
SCC       628
VASC      253
DF        239
Name: count, dtype: int64

Conjunto 1 - Train: 800 imágenes
clase
NV      100
MEL     100
BKL     100
DF      100
SCC     100
BCC     100
VASC    100
AK      100
Name: count, dtype: int64

Conjunto 1 - Test: 80 imágenes
clase
NV      10
MEL     10
BKL     10
DF      10
SCC     10
BCC     10
VASC    10
AK      10
Name: count, dtype: int64

Conjunto 2 - Train: 800 imágenes
clase
NV      100
MEL     100
BKL     100
DF      100
SCC     100
BCC     100
VASC    100
AK      100
Name: count, dtype: int64

Conjunto 2 - Test: 80 imágenes
clase
NV      10
MEL     10
BKL     10
DF      10
SCC     10
BCC     10
VASC    10
AK      10
Name: count, dtype: int64

✓ No hay solapamiento entre conjuntos

Total de imágenes a descargar: 1760


## 2. Descarga selectiva del ZIP

In [3]:
import zipfile
import io
import tempfile

zip_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Input.zip"

print("Descargando el archivo ZIP completo...")
print("(Esto puede tardar varios minutos, ~9-10 GB)\n")

# Descarga con streaming a archivo temporal
with tempfile.NamedTemporaryFile(delete=False, suffix='.zip') as tmp_file:
    tmp_path = Path(tmp_file.name)
    
    with requests.get(zip_url, stream=True, timeout=600) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        downloaded = 0
        
        for chunk in r.iter_content(chunk_size=1024*1024):  # 1 MB
            if chunk:
                tmp_file.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    pct = (downloaded / total_size) * 100
                    print(f"Descargado: {downloaded / (1024**3):.2f} GB / {total_size / (1024**3):.2f} GB ({pct:.1f}%)", end='\r')
    
    print("\n\n✓ Descarga completa. Extrayendo solo las 220 imágenes seleccionadas...\n")
    
    # Extrae SOLO las imágenes del subset
    extracted_count = 0
    not_found = []
    
    with zipfile.ZipFile(tmp_path, 'r') as z:
        all_files = z.namelist()
        
        for img_name in selected_images:
            # Busca el archivo en el zip (puede estar en una subcarpeta)
            matching = [f for f in all_files if img_name in f and f.endswith('.jpg')]
            
            if matching:
                for file_path in matching:
                    z.extract(file_path, dest_dir)
                    extracted_count += 1
                    if extracted_count % 20 == 0:
                        print(f"Extraídas: {extracted_count}/{len(selected_images)} imágenes", end='\r')
            else:
                not_found.append(img_name)
    
    
    if not_found:
        print(f"⚠ {len(not_found)} imágenes no encontradas en el ZIP:")
        print(not_found[:5], "..." if len(not_found) > 5 else "")

# Limpia el archivo temporal
tmp_path.unlink(missing_ok=True)

Descargando el archivo ZIP completo...
(Esto puede tardar varios minutos, ~9-10 GB)

Descargado: 9.10 GB / 9.10 GB (100.0%)

✓ Descarga completa. Extrayendo solo las 220 imágenes seleccionadas...

